# Estudo de Caso - Parte 1 (População vs Voto em Lula 2022)

### Configurações Iniciais


In [ ]:
!pip install geopandas

In [ ]:
!pip install pandas

In [ ]:
!pip install openpyxl

In [ ]:
!pip install xlrd

In [ ]:
!pip install matplotlib

In [ ]:
!pip install seaborn

In [ ]:
!pip install requests

In [ ]:
!pip install scikit-learn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests as req
from sklearn.linear_model import LinearRegression

### Tratamento de Dados

In [ ]:
data = pd.read_excel("data/estimativa_dou_2024.xls")

In [ ]:
data.head()

In [ ]:
data = pd.read_excel("data/estimativa_dou_2024.xls", header=1, sheet_name=1)

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data = data.drop(columns=["COD. UF", "COD. MUNIC"])

In [ ]:
data.head()

In [ ]:
# Remove rows with NaN in 'UF' or 'POPULAÇÃO ESTIMADA' and filter out summary/footer rows
filtered_data = data.dropna(subset=["UF", "POPULAÇÃO ESTIMADA"])
filtered_data = filtered_data[filtered_data["UF"].str.len() == 2]

# Create a color palette for each UF
ufs = filtered_data["UF"].unique()
palette = sns.color_palette("husl", len(ufs))
uf_color_map = dict(zip(ufs, palette))

plt.figure(figsize=(16, 12))
sns.boxplot(
    y="UF",
    x="POPULAÇÃO ESTIMADA",
    data=filtered_data,
    showcaps=True,
    boxprops={'facecolor':'None'},
    showfliers=False,
    whiskerprops={'linewidth':2},
    orient="h"
)
sns.stripplot(
    y="UF",
    x="POPULAÇÃO ESTIMADA",
    data=filtered_data,
    jitter=True,
    palette=uf_color_map,
    dodge=True,
    alpha=0.6,
    orient="h"
)
plt.yticks(rotation=0)
plt.title("Distribuição da População Estimada por UF")
plt.show()

In [ ]:
filtered_data.head()

In [ ]:
filtered_data.info()

In [ ]:
eleicoes = pd.read_excel("data/eleicoes_2022.xlsx")

In [ ]:
eleicoes.head()

In [ ]:
eleicoes.info()

In [ ]:
eleicoes.describe()

In [ ]:
print(eleicoes.columns.tolist())

In [ ]:
p_columns = [col for col in eleicoes.columns if col.startswith('PS22_1') and len(col) == 8]

In [ ]:
eleicoes_p = eleicoes[["CD_MUN_I", "MUN_NOME"] + p_columns].copy()
eleicoes_p.head()

In [ ]:
populacao = pd.read_excel("data/estimativa_dou_2024.xls", header=1, sheet_name=1)

In [ ]:
populacao.head()

In [ ]:
populacao['COD_COMBINED'] = (
    populacao['COD. UF'].fillna(0).astype(int).astype(str).str.zfill(2) +
    populacao['COD. MUNIC'].fillna(0).astype(int).astype(str).str.zfill(5)
)
populacao.head()

In [ ]:
populacao = populacao.drop(columns=["COD. UF", "COD. MUNIC"])

In [ ]:
# Remove COD_COMBINED after merge, keep only CD_MUN_I
populacao['COD_COMBINED'] = populacao['COD_COMBINED'].astype(str)
eleicoes_p['CD_MUN_I'] = eleicoes_p['CD_MUN_I'].astype(str)

merged_df = pd.merge(
    populacao,
    eleicoes_p,
    left_on="COD_COMBINED",
    right_on="CD_MUN_I",
    how="inner",
    validate="many_to_one"
)
merged_df.head()

In [ ]:
merged_df = merged_df.drop(columns=["COD_COMBINED"])

In [ ]:
# Ensure both keys are strings for merging
eleicoes_p['CD_MUN_I'] = eleicoes_p['CD_MUN_I'].astype(str)

merged_df = pd.merge(
    populacao,
    eleicoes_p,
    left_on="COD_COMBINED",
    right_on="CD_MUN_I",
    how="inner",
    validate="many_to_one"
)
merged_df.head()

In [ ]:
merged_df.info()

In [ ]:
ps22_1xx_columns = [col for col in merged_df.columns if col.startswith('PS22_1') and col not in ['PS22_1AB', 'PS22_1AP']]
merged_df['votos_totais'] = merged_df[ps22_1xx_columns].sum(axis=1)

In [ ]:
merged_df['voto_lula'] = (merged_df['PS22_113'] / merged_df['votos_totais']) * 100

In [ ]:
merged_df['voto_lula'].describe()

### Visualizações

In [ ]:
# Filtra apenas municípios com até 500.000 habitantes
filtered_df = merged_df[merged_df['POPULAÇÃO ESTIMADA'] <= 500000].copy()

# Use o restante do código normalmente com filtered_df
edges = np.concatenate([
    np.arange(0, 20000, 4000),         # 0-20k: bins every 2,000
    np.arange(20000, 50000, 6000),     # 20k-50k: bins every 5,000
    np.arange(50000, 100000, 5000),   # 50k-100k: bins every 10,000
    np.linspace(100000, 500000, 20)    # 10 bins from 100k to 500k
])
edges = np.unique(edges)

filtered_df['pop_bin'] = pd.cut(filtered_df['POPULAÇÃO ESTIMADA'], bins=edges, include_lowest=True)

binned_means = filtered_df.groupby('pop_bin').agg({
    'POPULAÇÃO ESTIMADA': 'mean',
    'voto_lula': 'mean'
}).dropna()

plt.figure(figsize=(12, 6))
sns.scatterplot(
    data=binned_means,
    x="POPULAÇÃO ESTIMADA",
    y="voto_lula",
    color="red",
    alpha=0.7
)
plt.title("População Estimada (média por bin) vs % Voto Lula (média por bin)")
plt.xlabel("População Estimada (média do bin)")
plt.ylabel("% Voto Lula (média do bin)")
plt.tight_layout()
plt.show()

In [ ]:
# Calcula a correlação de Pearson entre população estimada e % voto Lula
filtered_df = merged_df[merged_df['POPULAÇÃO ESTIMADA'] <= 500000].copy()
correlation = filtered_df[['POPULAÇÃO ESTIMADA', 'voto_lula']].corr(method='pearson').iloc[0,1]
print(f"Correlação de Pearson entre população estimada e % voto Lula: {correlation:.4f}")

In [ ]:
# Example: count number of cities in merged_df within custom population intervals
intervals = [(0, 5000), (5000, 10000), (10000, 20000), (20000, 50000), (50000, 100000), (100000, 500000), (500000, 1000000), (1000000, 5000000), (5000000, 12000000)]

counts = []
for low, high in intervals:
    count = merged_df[(merged_df['POPULAÇÃO ESTIMADA'] > low) & (merged_df['POPULAÇÃO ESTIMADA'] <= high)].shape[0]
    counts.append({'interval': f'({low}, {high}]', 'num_cities': count})

# Display result as DataFrame
pd.DataFrame(counts)

In [ ]:
# Use intervalos menores para popular mais o gráfico
custom_intervals = [
    (0, 2000), (2000, 4000), (4000, 6000), (6000, 8000), (8000, 10000),
    (10000, 15000), (15000, 20000), (20000, 30000), (30000, 40000), (40000, 50000),
    (50000, 75000), (75000, 100000), (100000, 150000), (150000, 200000), 
    (200000, 250000), (250000, 300000), (300000,500000), (500000, 1000000), (1000000, 12000000)
]
custom_edges = [low for low, high in custom_intervals] + [custom_intervals[-1][1]]

filtered_df['custom_bin'] = pd.cut(filtered_df['POPULAÇÃO ESTIMADA'], bins=custom_edges, include_lowest=True)

binned_means_custom = filtered_df.groupby('custom_bin').agg({
    'POPULAÇÃO ESTIMADA': 'median',
    'voto_lula': 'median'
}).dropna()

plt.figure(figsize=(12, 6))
sns.scatterplot(
    data=binned_means_custom,
    x="POPULAÇÃO ESTIMADA",
    y="voto_lula",
    color="red",
    alpha=0.7
)
plt.title("População Estimada vs Voto em Lula no primeiro turno de 2022")
plt.xlabel("População Estimada (média do intervalo)")
plt.ylabel("% Voto Lula (média do intervalo)")
plt.tight_layout()
# plt.savefig("scatter_populacao_lula.png", dpi=300) 
plt.show()

In [ ]:
binned_means_custom.reset_index().rename(columns={
    "POPULAÇÃO ESTIMADA": "População Estimada (mediana do intervalo)",
    "voto_lula": "% Voto Lula (mediana do intervalo)",
    "custom_bin": "Intervalo de População"
})

In [ ]:
# Intervalos mais granulares até 100.000 habitantes
granular_intervals = [
    (0, 2000), (2000, 4000), (4000, 6000), (6000, 8000), (8000, 10000),
    (10000, 12000), (12000, 14000), (14000, 16000), (16000, 18000), (18000, 20000),
    (20000, 25000), (25000, 30000), (30000, 35000), (35000, 40000), (40000, 45000),
    (45000, 50000), (50000, 60000), (60000, 70000), (70000, 80000), (80000, 90000), (90000, 100000),
    (100000, 150000), (150000, 200000), (200000, 250000), (250000, 300000), (300000, 500000),
    (500000, 1000000), (1000000, 12000000)
]
granular_edges = [low for low, high in granular_intervals] + [granular_intervals[-1][1]]

filtered_df['granular_bin'] = pd.cut(filtered_df['POPULAÇÃO ESTIMADA'], bins=granular_edges, include_lowest=True)

table = filtered_df.groupby('granular_bin').agg(
    POPULAÇÃO_MEDIANA=('POPULAÇÃO ESTIMADA', 'median'),
    VOTO_LULA_MEDIANA=('voto_lula', 'median')
).dropna().reset_index()

table

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(table['POPULAÇÃO_MEDIANA'], table['VOTO_LULA_MEDIANA'], color='red', alpha=0.7)
plt.xlabel("População Estimada (mediana do intervalo)")
plt.ylabel("% Voto Lula (mediana do intervalo)")
plt.title("População Estimada vs % Voto Lula (medianas por intervalo granular)")
plt.tight_layout()
plt.xlim(right=100000)
plt.show()

# Parte 2 - Análises por região / UF

In [ ]:
# Define mapping from UF to Região
uf_to_regiao = {
    'RO': 'Norte', 'AC': 'Norte', 'AM': 'Norte', 'RR': 'Norte', 'PA': 'Norte', 'AP': 'Norte', 'TO': 'Norte',
    'MA': 'Nordeste', 'PI': 'Nordeste', 'CE': 'Nordeste', 'RN': 'Nordeste', 'PB': 'Nordeste', 'PE': 'Nordeste', 'AL': 'Nordeste', 'SE': 'Nordeste', 'BA': 'Nordeste',
    'MG': 'Sudeste', 'ES': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'SC': 'Sul', 'RS': 'Sul',
    'MS': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'DF': 'Centro-Oeste'
}

# Add 'região' column to merged_df using the UF column
merged_df['região'] = merged_df['UF'].map(uf_to_regiao)

In [ ]:
merged_df['região'].value_counts()

In [ ]:
# Scatter plot de População Estimada (mediana do intervalo) vs % Voto Lula (mediana do intervalo) por região, em gráficos separados

# Adiciona a coluna 'região' ao filtered_df
filtered_df['região'] = filtered_df['UF'].map(uf_to_regiao)

# Cria um DataFrame com a mediana por região e bin
region_bins = (
    filtered_df
    .groupby(['região', 'custom_bin'])
    .agg({'POPULAÇÃO ESTIMADA': 'median', 'voto_lula': 'median'})
    .reset_index()
    .dropna()
)

regioes = region_bins['região'].unique()
num_regioes = len(regioes)
fig, axes = plt.subplots(nrows=(num_regioes + 1) // 2, ncols=2, figsize=(16, 4 * ((num_regioes + 1) // 2)), sharex=True, sharey=True)

for ax, regiao in zip(axes.flat, regioes):
    dados = region_bins[region_bins['região'] == regiao]
    ax.scatter(
        dados['POPULAÇÃO ESTIMADA'],
        dados['voto_lula'],
        color='tab:red',
        alpha=0.8,
        s=100
    )
    ax.set_title(f"{regiao}")
    ax.set_xlabel("População Estimada (mediana do intervalo)")
    ax.set_ylabel("% Voto Lula (mediana do intervalo)")
    ax.grid(True)

# Remove subplots extras se houver
for i in range(len(regioes), axes.size):
    fig.delaxes(axes.flat[i])

plt.suptitle("População Estimada vs % Voto Lula - Separado por Região", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.97])
# plt.savefig("scatter_populacao_lula_regiao.png", dpi=300)
plt.show()

In [ ]:
# Gráfico combinado: População Estimada (mediana do intervalo) vs % Voto Lula (mediana do intervalo) para todas as regiões no mesmo gráfico

plt.figure(figsize=(12, 7))
for regiao in regioes:
    dados_regiao = region_bins[region_bins['região'] == regiao]
    plt.scatter(
        dados_regiao['POPULAÇÃO ESTIMADA'],
        dados_regiao['voto_lula'],
        label=regiao,
        s=100,
        alpha=0.8
    )

plt.xlabel("População Estimada (mediana do intervalo)")
plt.ylabel("% Voto Lula (mediana do intervalo)")
plt.title("População Estimada vs % Voto Lula (medianas por intervalo) - Todas as Regiões")
plt.legend(title="Região")
plt.tight_layout()
# plt.savefig("scatter_populacao_lula_todas_regioes.png", dpi=300)
plt.show()

In [ ]:
# Filtra os dados para SP e MG
ufs_to_plot = ['SP', 'MG']
df_uf = filtered_df[filtered_df['UF'].isin(ufs_to_plot)]

# Agrupa por UF e granular_bin, calcula a mediana da população e do voto em Lula
uf_bins = (
    df_uf
    .groupby(['UF', 'granular_bin'])
    .agg({'POPULAÇÃO ESTIMADA': 'median', 'voto_lula': 'median'})
    .reset_index()
    .dropna()
)

plt.figure(figsize=(10, 6))
for uf in ufs_to_plot:
    dados = uf_bins[uf_bins['UF'] == uf]
    plt.scatter(
        dados['POPULAÇÃO ESTIMADA'],
        dados['voto_lula'],
        label=uf,
        alpha=0.8,
        s=80
    )

plt.xlabel("População Estimada (mediana do intervalo)")
plt.ylabel("% Voto Lula (mediana do intervalo)")
plt.title("População Estimada vs % Voto Lula (medianas por bin) - SP e MG")
plt.legend(title="UF")
plt.tight_layout()
# plt.savefig("scatter_populacao_lula_sp_mg.png", dpi=300)
plt.show()

In [ ]:
# Filtra os dados para RS e PE
ufs_to_plot_rs_pe = ['RS', 'PE']
df_uf_rs_pe = filtered_df[filtered_df['UF'].isin(ufs_to_plot_rs_pe)]

# Agrupa por UF e granular_bin, calcula a mediana da população e do voto em Lula
df_bins_uf_rs_pe = (
    df_uf_rs_pe
    .groupby(['UF', 'granular_bin'])
    .agg({'POPULAÇÃO ESTIMADA': 'median', 'voto_lula': 'median'})
    .reset_index()
    .dropna()
)

plt.figure(figsize=(10, 6))
for uf in ufs_to_plot_rs_pe:
    dados = df_bins_uf_rs_pe[df_bins_uf_rs_pe['UF'] == uf]
    plt.scatter(
        dados['POPULAÇÃO ESTIMADA'],
        dados['voto_lula'],
        label=uf,
        alpha=0.8,
        s=80
    )

plt.xlabel("População Estimada (mediana do intervalo)")
plt.ylabel("% Voto Lula (mediana do intervalo)")
plt.title("População Estimada vs % Voto Lula (medianas por bin) - RS e PE")
plt.legend(title="UF")
plt.tight_layout()
# plt.savefig("scatter_populacao_lula_rs_pe.png", dpi=300)
plt.show()

In [ ]:
# Calcula a correlação de Pearson entre população estimada e % voto Lula para cada região
correlations = (
    filtered_df.groupby('região')
    .apply(lambda g: g[['POPULAÇÃO ESTIMADA', 'voto_lula']].corr(method='pearson').iloc[0,1])
    .rename('correlação')
    .reset_index()
)
print(correlations)

In [ ]:
# Calcula a correlação de Pearson entre população estimada e % voto Lula para cada UF
correlations_uf = (
    filtered_df.groupby('UF')
    .apply(lambda g: pd.Series({
        'correlação': g[['POPULAÇÃO ESTIMADA', 'voto_lula']].corr(method='pearson').iloc[0,1],
        'num_municipios': len(g)
    }))
    .reset_index()
)
print(correlations_uf)

# Parte 3 - Analisando Renda per Capita

## Adicionando base de renda per capita

In [ ]:
# Use the code from cell 57 to load the correct data
renda = pd.read_csv("data/renda_per_capita_ibge.csv", sep = ";")

In [ ]:
renda.head()

In [ ]:
renda = renda.drop(columns=['Nível'])

In [ ]:
renda = renda.rename(columns={'2021': 'pib per capita'})

In [ ]:
# Split 'Município' into 'Município' and 'UF' columns in the 'renda' DataFrame
renda[['Município', 'UF']] = renda['Município'].str.extract(r'^(.*) \((\w{2})\)$')

In [ ]:
renda.head()

In [ ]:
renda = renda[['Cód.', 'pib per capita']].copy()
renda.head()

In [ ]:
# Ensure both columns are strings for merging
renda['Cód.'] = renda['Cód.'].astype(str)
merged_df['CD_MUN_I'] = merged_df['CD_MUN_I'].astype(str)

# Merge PIB per capita into merged_df
merged_df = pd.merge(
    merged_df,
    renda[['Cód.', 'pib per capita']],
    left_on='CD_MUN_I',
    right_on='Cód.',
    how='left'
).drop(columns=['Cód.'])

# Show result
merged_df.head()

In [ ]:
# Corrige a coluna 'pib per capita' para ser realmente o valor per capita
merged_df['pib_per_capita_real'] = merged_df['pib per capita'] / merged_df['POPULAÇÃO ESTIMADA']

# Exibe as primeiras linhas para conferência
merged_df[['NOME DO MUNICÍPIO', 'pib per capita', 'POPULAÇÃO ESTIMADA', 'pib_per_capita_real']].head()

In [ ]:
merged_df = merged_df.drop(columns=['pib per capita'])
merged_df = merged_df.rename(columns={'pib_per_capita_real': 'pib per capita'})
merged_df['pib per capita'] = merged_df['pib per capita'] * 1000

## Análises com renda per capita

In [ ]:
# Bin the population into intervals and plot median PIB per capita per bin
df_500k = merged_df[merged_df['POPULAÇÃO ESTIMADA'] <= 500000].copy()
df_500k['pop_bin'] = pd.cut(df_500k['POPULAÇÃO ESTIMADA'], bins=granular_edges, include_lowest=True)

binned_pib = df_500k.groupby('pop_bin').agg({
    'POPULAÇÃO ESTIMADA': 'median',
    'pib per capita': 'median'
}).dropna()

plt.figure(figsize=(10, 6))
plt.scatter(binned_pib['POPULAÇÃO ESTIMADA'], binned_pib['pib per capita'], alpha=0.6, color = 'red')
plt.xlabel("População Estimada (mediana do bin)")
plt.ylabel("PIB per capita (mediana do bin)")
plt.title("População Estimada vs PIB per capita (medianas por bin, até 500k habitantes)")
# plt.savefig("scatter_populacao_pib.png", dpi=300)
plt.tight_layout()
plt.show()

In [ ]:
# Plot both the table from cell 42 and the binned_pib from cell 67 in the same figure

fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot % Voto Lula (mediana do intervalo) vs População Estimada (mediana do intervalo)
ax1.scatter(table['POPULAÇÃO_MEDIANA'], table['VOTO_LULA_MEDIANA'], color='red', alpha=0.7, label='% Voto Lula (mediana)')
ax1.set_xlabel("População Estimada (mediana do intervalo)")
ax1.set_ylabel("% Voto Lula (mediana do intervalo)", color='red')
ax1.tick_params(axis='y', labelcolor='red')

# Create a second y-axis for PIB per capita
ax2 = ax1.twinx()
ax2.scatter(binned_pib['POPULAÇÃO ESTIMADA'], binned_pib['pib per capita'], color='blue', alpha=0.6, label='PIB per capita (mediana)')
ax2.set_ylabel("PIB per capita (mediana do intervalo)", color='blue')
ax2.tick_params(axis='y', labelcolor='blue')

plt.title("População Estimada vs % Voto Lula e PIB per capita (medianas por intervalo)")
fig.tight_layout()
# plt.savefig("scatter_populacao_voto_pib.png", dpi=300)
plt.show()

In [ ]:
# Calcula a correlação de Pearson entre PIB per capita e População Estimada
cor_pib_pop = merged_df[['pib per capita', 'POPULAÇÃO ESTIMADA']].corr(method='pearson').iloc[0, 1]
print(f"Correlação de Pearson entre PIB per capita e População Estimada: {cor_pib_pop:.4f}")

In [ ]:
# Use more granular bins for PIB per capita to visualize the relationship with % Voto Lula
df_pib400 = merged_df[merged_df['pib per capita'] <= 400000].copy()
bins = np.linspace(df_pib400['pib per capita'].min(), df_pib400['pib per capita'].max(), 100)
df_pib400['pib_bin'] = pd.cut(df_pib400['pib per capita'], bins=bins, include_lowest=True)

binned = df_pib400.groupby('pib_bin').agg({
    'pib per capita': 'median',
    'voto_lula': 'median'
}).dropna()

plt.figure(figsize=(10, 6))
plt.scatter(binned['pib per capita'], binned['voto_lula'], alpha=0.7, color = 'red')
plt.xlabel("PIB per capita")
plt.ylabel("% Voto Lula")
plt.title("PIB per capita vs % Voto Lula (municípios)")
plt.tight_layout()
# plt.savefig("scatter_pib_voto_lula.png", dpi=300)
plt.show()

In [ ]:
# Calcula a correlação de Pearson entre % voto Lula e PIB per capita
cor_lula_pib = merged_df[['voto_lula', 'pib per capita']].corr(method='pearson').iloc[0, 1]
print(f"Correlação de Pearson entre % voto Lula e PIB per capita: {cor_lula_pib:.4f}")

In [ ]:
# Cria os mesmos bins usados em cell 71
bins = np.linspace(merged_df['pib per capita'].min(), 400000, 100)
merged_df['pib_bin_400'] = pd.cut(merged_df['pib per capita'], bins=bins, include_lowest=True)

# Agrupa e calcula as medianas
table_lula_pib_400 = merged_df.groupby('pib_bin_400').agg({
    'pib per capita': 'median',
    'voto_lula': 'median'
}).rename(columns={
    'pib per capita': 'PIB per capita Mediano',
    'voto_lula': '% Voto Lula Mediano'
}).reset_index().rename(columns={'pib_bin_400': 'Intervalo PIB per capita'})

# Exibe apenas as colunas desejadas
table_lula_pib_400[['PIB per capita Mediano', '% Voto Lula Mediano']].dropna()

In [ ]:
# Restrict to PIB per capita below 50 and plot PIB per capita vs % Voto Lula (medianas por intervalo)

# Filter merged_df for PIB per capita < 50
df_below_50 = merged_df[merged_df['pib per capita'] < 50000].copy()

# Bin PIB per capita into intervals
bins = np.linspace(df_below_50['pib per capita'].min(), df_below_50['pib per capita'].max(), 20)
df_below_50['pib_bin'] = pd.cut(df_below_50['pib per capita'], bins=bins, include_lowest=True)

# Group by PIB bin and calculate medians
table_pib_below_50 = df_below_50.groupby('pib_bin').agg(
    PIB_PER_CAPITA_MEDIANA=('pib per capita', 'median'),
    VOTO_LULA_MEDIANA=('voto_lula', 'median')
).dropna().reset_index()

plt.figure(figsize=(10, 6))
plt.scatter(table_pib_below_50['PIB_PER_CAPITA_MEDIANA'], table_pib_below_50['VOTO_LULA_MEDIANA'], color='red', alpha=0.7)
plt.xlabel("PIB per capita (mediana do intervalo)")
plt.ylabel("% Voto Lula (mediana do intervalo)")
plt.title("PIB per capita vs % Voto Lula (PIB per capita < 50)")
plt.tight_layout()
# plt.savefig("scatter_pib_below_50_voto_lula.png", dpi=300)
plt.show()

In [ ]:
# Scatter plot: PIB per capita vs % Voto Lula, separado por região (usando bins)
region_pib_bins = (
    merged_df
    .dropna(subset=['pib_bin_400', 'região'])
    .groupby(['região', 'pib_bin_400'])
    .agg({'pib per capita': 'median', 'voto_lula': 'median'})
    .reset_index()
    .dropna()
)

regioes = region_pib_bins['região'].unique()
num_regioes = len(regioes)
fig, axes = plt.subplots(nrows=(num_regioes + 1) // 2, ncols=2, figsize=(16, 4 * ((num_regioes + 1) // 2)), sharex=True, sharey=True)

for ax, regiao in zip(axes.flat, regioes):
    dados = region_pib_bins[region_pib_bins['região'] == regiao]
    ax.scatter(
        dados['pib per capita'],
        dados['voto_lula'],
        color='red',
        alpha=0.8,
        s=100,
    )
    ax.set_title(f"{regiao}")
    ax.set_xlabel("PIB per capita (mediana do bin)")
    ax.set_ylabel("% Voto Lula (mediana do bin)")
    ax.grid(True)

# Remove subplots extras se houver
for i in range(len(regioes), axes.size):
    fig.delaxes(axes.flat[i])

plt.suptitle("PIB per capita vs % Voto Lula - Separado por Região", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()


# Parte 4 - Analisando Bolsa Família

In [ ]:
import pandas as pd

# Load the existing Bolsa Família CSV file (already downloaded)
print("Carregando dados do Bolsa Família do arquivo local...")

# Carregar os dados em um DataFrame
bolsa_familia_raw = pd.read_csv("data/bolsa_familia_2025.csv")
print(f"Dataset raw carregado com {len(bolsa_familia_raw)} registros")

# FIX 1: Handle multiple years - get most recent data per municipality
print("\nProcessando dados (removendo duplicatas por município)...")
bolsa_familia_unique = bolsa_familia_raw.groupby('codigo_ibge')['qtd_familias_beneficiarias_bolsa_familia_s'].max().reset_index()
print(f"Depois de remover duplicatas: {len(bolsa_familia_unique)} municípios únicos")

# FIX 2: Keep Bolsa Família codes as they are (6 digits) - we'll truncate our IBGE codes instead
print("Mantendo códigos IBGE originais da API (6 dígitos)...")

# Keep only the necessary columns for merging
bolsa_familia = bolsa_familia_unique[['codigo_ibge', 'qtd_familias_beneficiarias_bolsa_familia_s']].copy()

print(f"\nDataset final processado:")
print(f"- {len(bolsa_familia)} municípios")
print(f"- Códigos IBGE mantidos no formato original (6 dígitos)")
print(f"- Dados mais recentes por município")

# Show sample of processed data
print(f"\nPrimeiras linhas do dataset processado:")
print(bolsa_familia.head())

# Show code format comparison
print(f"\nFormato dos códigos:")
print(f"Bolsa Família API: {bolsa_familia_raw['codigo_ibge'].iloc[0]} (6 dígitos)")
print(f"Processado: {bolsa_familia['codigo_ibge'].iloc[0]} (6 dígitos)")
# Data is already processed in the download cell - showing processed data
bolsa_familia.head()

In [ ]:
# Truncate COD_COMBINED to 6 digits for merging
merged_df['COD_COMBINED_6'] = merged_df['COD_COMBINED'].str[:6]
bolsa_familia['codigo_ibge'] = bolsa_familia['codigo_ibge'].astype(str)

# Merge only qtd_familias_beneficiarias_bolsa_familia_s into merged_df
merged_df = merged_df.merge(
    bolsa_familia[['codigo_ibge', 'qtd_familias_beneficiarias_bolsa_familia_s']],
    left_on='COD_COMBINED_6',
    right_on='codigo_ibge',
    how='left'
)

# Optionally drop helper columns if not needed
merged_df = merged_df.drop(columns=['COD_COMBINED_6', 'codigo_ibge'])

In [ ]:
merged_df['perc_bolsa_familia'] = (merged_df['qtd_familias_beneficiarias_bolsa_familia_s'] / merged_df['POPULAÇÃO ESTIMADA']) * 100

merged_df.head()

In [ ]:
merged_df.head()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(merged_df['perc_bolsa_familia'], merged_df['voto_lula'], alpha=0.7, color='red', s=20)
plt.xlabel('% de Famílias Beneficiárias do Bolsa Família')
plt.ylabel('% Voto Lula')
plt.title('Relação entre % Bolsa Família e % Voto Lula (municípios)')
plt.tight_layout()
#plt.savefig("scatter_bolsa_familia_voto_lula.png", dpi=300)
plt.show()


In [ ]:
# Calculate the Pearson correlation between % de Famílias Beneficiárias do Bolsa Família and % Voto Lula
if 'perc_bolsa_familia' in merged_df.columns:
    correlation_bf_lula = merged_df[['perc_bolsa_familia', 'voto_lula']].corr(method='pearson').iloc[0, 1]
    print(f"Correlação de Pearson entre % Bolsa Família e % Voto Lula: {correlation_bf_lula:.4f}")
else:
    print("Column 'perc_bolsa_familia' not found in merged_df.")

In [ ]:
# Scatter plot: % de Famílias Beneficiárias do Bolsa Família vs % Voto Lula, separado por região

if 'perc_bolsa_familia' in merged_df.columns and 'região' in merged_df.columns:
    regioes = merged_df['região'].dropna().unique()
    num_regioes = len(regioes)
    fig, axes = plt.subplots(nrows=(num_regioes + 1) // 2, ncols=2, figsize=(16, 4 * ((num_regioes + 1) // 2)), sharex=True, sharey=True)

    for ax, regiao in zip(axes.flat, regioes):
        dados = merged_df[merged_df['região'] == regiao]
        ax.scatter(
            dados['perc_bolsa_familia'],
            dados['voto_lula'],
            color='red',
            alpha=0.7,
            s=30
        )
        ax.set_title(f"{regiao}")
        ax.set_xlabel("% Famílias Bolsa Família")
        ax.set_ylabel("% Voto Lula")
        ax.grid(True)

    # Remove subplots extras se houver
    for i in range(len(regioes), axes.size):
        fig.delaxes(axes.flat[i])

    plt.suptitle("% Bolsa Família vs % Voto Lula - Separado por Região", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    #plt.savefig("scatter_bolsa_familia_voto_lula_regiao.png", dpi=300)
    plt.show()
else:
    print("Columns 'perc_bolsa_familia' or 'região' not found in merged_df.")

In [ ]:
# Calculate Pearson correlation between % Bolsa Família and % Voto Lula per region
if 'perc_bolsa_familia' in merged_df.columns and 'região' in merged_df.columns:
    corr_per_region = (
        merged_df.groupby('região')
        .apply(lambda g: g[['perc_bolsa_familia', 'voto_lula']].corr(method='pearson').iloc[0, 1])
        .rename('correlação')
        .reset_index()
    )
    print(corr_per_region)
else:
    print("Columns 'perc_bolsa_familia' or 'região' not found in merged_df.")

## Por que a correlação geral é maior que as correlações regionais?

Este é um exemplo clássico do **Paradoxo de Simpson** - quando uma tendência aparece em grupos combinados, mas desaparece ou se inverte quando os grupos são examinados separadamente.

### Possíveis explicações:

1. **Diferenças estruturais entre regiões**: Regiões com maior % Bolsa Família podem naturalmente ter maior % voto Lula
2. **Variáveis confundidoras**: Fatores socioeconômicos, históricos ou culturais que afetam ambas as variáveis
3. **Heterogeneidade regional**: Cada região pode ter dinâmicas políticas e sociais distintas

In [ ]:
# Vamos investigar o Paradoxo de Simpson com estatísticas descritivas por região

if 'perc_bolsa_familia' in merged_df.columns and 'região' in merged_df.columns:
    print("=== ANÁLISE DO PARADOXO DE SIMPSON ===\n")
    
    # Correlação geral
    cor_geral = merged_df[['perc_bolsa_familia', 'voto_lula']].corr(method='pearson').iloc[0, 1]
    print(f"Correlação GERAL: {cor_geral:.4f}\n")
    
    # Estatísticas por região
    stats_regiao = merged_df.groupby('região').agg({
        'perc_bolsa_familia': ['mean', 'std', 'count'],
        'voto_lula': ['mean', 'std']
    }).round(2)
    
    print("Estatísticas descritivas por região:")
    print(stats_regiao)
    print("\n" + "="*80 + "\n")
    
    # Correlações por região (repetindo para clareza)
    print("Correlações por região:")
    corr_per_region = (
        merged_df.groupby('região')
        .apply(lambda g: g[['perc_bolsa_familia', 'voto_lula']].corr(method='pearson').iloc[0, 1])
        .rename('correlação')
        .reset_index()
    )
    for _, row in corr_per_region.iterrows():
        print(f"{row['região']}: {row['correlação']:.4f}")
    
    print(f"\nMédia das correlações regionais: {corr_per_region['correlação'].mean():.4f}")
    print(f"Diferença (Geral - Média Regional): {cor_geral - corr_per_region['correlação'].mean():.4f}")
else:
    print("Colunas necessárias não encontradas.")

In [ ]:
# Visualização do Paradoxo de Simpson: médias regionais vs tendência geral

if 'perc_bolsa_familia' in merged_df.columns and 'região' in merged_df.columns:
    # Calcular médias por região
    medias_regiao = merged_df.groupby('região').agg({
        'perc_bolsa_familia': 'mean',
        'voto_lula': 'mean',
        'POPULAÇÃO ESTIMADA': 'sum'  # população total por região
    }).reset_index()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Gráfico 1: Todos os municípios (correlação geral)
    ax1.scatter(merged_df['perc_bolsa_familia'], merged_df['voto_lula'], 
                alpha=0.3, color='lightblue', s=10, label='Municípios')
    
    # Linha de tendência geral
    z = np.polyfit(merged_df['perc_bolsa_familia'].dropna(), 
                   merged_df['voto_lula'][merged_df['perc_bolsa_familia'].notna()], 1)
    p = np.poly1d(z)
    x_trend = np.linspace(merged_df['perc_bolsa_familia'].min(), 
                          merged_df['perc_bolsa_familia'].max(), 100)
    ax1.plot(x_trend, p(x_trend), "r--", alpha=0.8, label='Tendência Geral')
    
    ax1.set_xlabel('% Famílias Bolsa Família')
    ax1.set_ylabel('% Voto Lula')
    ax1.set_title(f'Correlação Geral\nr = {cor_geral:.4f}')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Gráfico 2: Médias por região (mostra o efeito regional)
    colors = ['red', 'blue', 'green', 'orange', 'purple']
    for i, regiao in enumerate(medias_regiao['região']):
        dados_regiao = merged_df[merged_df['região'] == regiao]
        ax2.scatter(dados_regiao['perc_bolsa_familia'], dados_regiao['voto_lula'], 
                    alpha=0.4, s=15, label=regiao, color=colors[i % len(colors)])
        
        # Marcar a média da região
        media_bf = medias_regiao[medias_regiao['região'] == regiao]['perc_bolsa_familia'].iloc[0]
        media_lula = medias_regiao[medias_regiao['região'] == regiao]['voto_lula'].iloc[0]
        ax2.scatter(media_bf, media_lula, s=200, color=colors[i % len(colors)], 
                    marker='*', edgecolor='black', linewidth=1)
    
    # Linha conectando médias regionais
    ax2.plot(medias_regiao['perc_bolsa_familia'], medias_regiao['voto_lula'], 
             'k--', alpha=0.7, linewidth=2, label='Médias Regionais')
    
    ax2.set_xlabel('% Famílias Bolsa Família')
    ax2.set_ylabel('% Voto Lula')
    ax2.set_title('Separado por Região\n(estrelas = médias regionais)')
    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    #plt.savefig("simpson_paradox_bolsa_familia.png", dpi=300)
    plt.show()
    
    print("\nMédias por região:")
    print(medias_regiao.round(2))
else:
    print("Colunas necessárias não encontradas.")

# Salvando o dataframe para uso posterior

In [ ]:
# Save merged_df to pickle file for sharing with other notebooks
merged_df.to_pickle("data/merged_df.pkl")
print(f"DataFrame salvo em 'data/merged_df.pkl'")
print(f"Shape: {merged_df.shape}")
print(f"Colunas: {list(merged_df.columns)}")

In [ ]:
# Alternative: Save to CSV (human-readable but slower)
# merged_df.to_csv("data/merged_df.csv", index=False)
# print("DataFrame também salvo em 'data/merged_df.csv'")

## Como usar no outro notebook:

### Para carregar do pickle:
```python
import pandas as pd
merged_df = pd.read_pickle("data/merged_df.pkl")
print(f"DataFrame carregado: {merged_df.shape}")
```

### Para carregar do CSV:
```python
import pandas as pd
merged_df = pd.read_csv("data/merged_df.csv")
print(f"DataFrame carregado: {merged_df.shape}")
```

### Verificar se o arquivo existe:
```python
import os
if os.path.exists("data/merged_df.pkl"):
    print("Arquivo pickle encontrado!")
else:
    print("Execute o notebook parte1-4 primeiro para gerar os dados.")
```